In [48]:
import pandas as pd
import numpy as np
import time as t
from pathlib import Path
from xlwings import view

In [25]:
time_stamp = t.time()

In [26]:
# Source files
file_path_src_pq            = Path("D:/Analytik/3_ФТТ/", "_src_pq.xlsx")
file_path_src_ftt           = Path("D:/Analytik/3_ФТТ/_исх/2023/2/", "фтт.xlsx")
file_path_src_ttzz_squ      = Path("D:/Analytik/3_ФТТ/_исх/", "ттзз_скю.xlsx")

# Manuals dataframe
shimpments_qnt_df           = pd.read_excel(file_path_src_pq, "Количество отгрузок")
departments_df              = pd.read_excel(file_path_src_pq, "Справочник отделов")

# Source dataframe
src_ftt_df                  = pd.read_excel(file_path_src_ftt, header=[0, 1])

# Amounting month
month                       = t.strftime("01.02.2023")

c:\Users\derachic_ai\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [27]:
# Количество отгрузок
shimpments_qnt_df = shimpments_qnt_df.loc[shimpments_qnt_df["Начало месяца"] == month]
shimpments_qnt_df.columns = ["Отдел", "Начало месяца", "Значение", "Атрибут"]

# Преобразование object в date
shimpments_qnt_df['Начало месяца'] = pd.to_datetime(shimpments_qnt_df['Начало месяца'], format='%d.%m.%Y')
shimpments_qnt_df

,Отдел,Начало месяца,Значение,Атрибут
9,Главмолоко,2023-02-01,24,Количество отгрузок
10,ФЕРМА,2023-02-01,24,Количество отгрузок
11,VIP,2023-02-01,24,Количество отгрузок


In [28]:
# Сводную таблицу в плоскую
id_vars = src_ftt_df.columns[0:5].to_list()
value_vars = src_ftt_df.columns[5:].to_list()
df = src_ftt_df.melt(id_vars=id_vars, value_vars=value_vars, value_name='Значение', var_name='Дата').dropna()

# Преобразование object в date
df['Дата'] = pd.to_datetime(df['Дата'], format='%d.%m.%y')

# Список полей для переименования
columns = []
for x in df.columns[0:5]:
    columns.append(x[1])    # берем заголовки первого уровня
for x in df.columns[5:]:
    columns.append(x)    

df.columns = columns

df = pd.merge(df, departments_df, on=["Отдел продаж", "Подразделение"])
df["Начало месяца"] = pd.to_datetime(df['Дата']).dt.to_period('M').dt.to_timestamp()

In [29]:
# Продажи, руб.
sales = df.groupby(["Отдел", "Начало месяца"]).sum()
sales["Атрибут"] = "Продажи, руб."
sales


,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2023-02-01,5066844.88,"Продажи, руб."
Главмолоко,2023-02-01,39351827.66,"Продажи, руб."
ФЕРМА,2023-02-01,30028787.53,"Продажи, руб."


In [30]:
# Средняя отгрузка
avg_shimpment_df = pd.merge(sales, shimpments_qnt_df, on=["Отдел", "Начало месяца"])
avg_shimpment_df["Значение"] = avg_shimpment_df["Значение_x"] / avg_shimpment_df["Значение_y"]
avg_shimpment_df.drop(columns=["Значение_x", "Значение_y", "Атрибут_x", "Атрибут_y"], inplace=True)
avg_shimpment_df["Атрибут"] = "Средняя отгрузка"
avg_shimpment_df

,Отдел,Начало месяца,Значение,Атрибут
0,VIP,2023-02-01,2.111185e+05,Средняя отгрузка
1,Главмолоко,2023-02-01,1.639659e+06,Средняя отгрузка
2,ФЕРМА,2023-02-01,1.251199e+06,Средняя отгрузка


In [31]:
# Количество ТРТ
qnt_rtt = df.groupby(["Отдел", "Начало месяца"])["РТТ"].nunique()
qnt_rtt = qnt_rtt.to_frame(name="Значение")
qnt_rtt["Атрибут"] = "Количество ТРТ"
qnt_rtt


,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2023-02-01,83,Количество ТРТ
Главмолоко,2023-02-01,2466,Количество ТРТ
ФЕРМА,2023-02-01,2415,Количество ТРТ


In [32]:
# Количество должностей
qnt_posts = df.groupby(["Отдел", "Начало месяца"])["Должность ТА"].nunique()
qnt_posts = qnt_posts.to_frame(name="Значение")
qnt_posts["Атрибут"] = "Количество должностей"
qnt_posts

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2023-02-01,8,Количество должностей
Главмолоко,2023-02-01,29,Количество должностей
ФЕРМА,2023-02-01,23,Количество должностей


In [33]:
# Количество ТА
qnt_ta = df.groupby(["Отдел", "Начало месяца"])["ФИО ТА"].nunique()
qnt_ta = qnt_ta.to_frame(name="Значение")
qnt_ta["Атрибут"] = "Количество ТА"
qnt_ta

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2023-02-01,6,Количество ТА
Главмолоко,2023-02-01,30,Количество ТА
ФЕРМА,2023-02-01,22,Количество ТА


In [34]:
# Количество заказов
qnt_orders = df[["Отдел", "Подразделение", "Начало месяца", "РТТ", "Дата"]].drop_duplicates()
qnt_orders = qnt_orders.groupby(["Отдел", "Начало месяца"])["РТТ"].count()
qnt_orders = qnt_orders.to_frame(name="Значение")
qnt_orders["Атрибут"] = "Количество заказов"
qnt_orders

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2023-02-01,1099,Количество заказов
Главмолоко,2023-02-01,15398,Количество заказов
ФЕРМА,2023-02-01,13225,Количество заказов


In [35]:
# Среднее кол-во заказов в ТРТ
avg_orders_qnt = pd.merge(qnt_orders, qnt_rtt, on=["Отдел", "Начало месяца"])
avg_orders_qnt["Значение"] = avg_orders_qnt["Значение_x"]/avg_orders_qnt["Значение_y"]
avg_orders_qnt.drop(columns=["Значение_x", "Атрибут_x", "Значение_y", "Атрибут_y"], inplace=True)
avg_orders_qnt["Атрибут"] = "Среднее кол-во заказов в ТРТ"
avg_orders_qnt

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2023-02-01,13.240964,Среднее кол-во заказов в ТРТ
Главмолоко,2023-02-01,6.244120,Среднее кол-во заказов в ТРТ
ФЕРМА,2023-02-01,5.476190,Среднее кол-во заказов в ТРТ


In [36]:
# Среднее кол-во ТРТ в день на должность
avq_rtt_day_post = qnt_orders.merge(shimpments_qnt_df, on=["Отдел", "Начало месяца"]).merge(qnt_posts, on=["Отдел", "Начало месяца"])
avq_rtt_day_post["Значение"] = avq_rtt_day_post["Значение_x"]/avq_rtt_day_post["Значение_y"]/avq_rtt_day_post["Значение"]
avq_rtt_day_post.drop(columns=["Значение_x", "Атрибут_x", "Значение_y", "Атрибут_y"], inplace=True)
avq_rtt_day_post["Атрибут"] = "Среднее кол-во ТРТ в день на должность"
avq_rtt_day_post

,Отдел,Начало месяца,Значение,Атрибут
0,VIP,2023-02-01,5.723958,Среднее кол-во ТРТ в день на должность
1,Главмолоко,2023-02-01,22.123563,Среднее кол-во ТРТ в день на должность
2,ФЕРМА,2023-02-01,23.958333,Среднее кол-во ТРТ в день на должность


In [56]:
# Количество ТРТ с грппировкой по ЗЗ
qnt_orders = df[["Отдел", "Подразделение", "Начало месяца", "РТТ", "Дата"]].drop_duplicates()
qnt_orders = qnt_orders.groupby(["Отдел", "Начало месяца", "РТТ"])["Дата"].count()
qnt_orders = qnt_orders.to_frame(name="Значение")
qnt_orders["Атрибут"] = "Количество заказов"
qnt_orders["Атрибут1"] = np.where(qnt_orders["Значение"] < 3, "1-2 разовых заказов за месяц",
                                  np.where(qnt_orders["Значение"] < 5, "3-4 разовых заказов за месяц",
                                           np.where(qnt_orders["Значение"] < 8, "5-7 разовых заказов за месяц", 
                                                    np.where(qnt_orders["Значение"] > 8, "8 и выше заказов за месяц", 0)
                                                    )
                                            )
                                  )
qnt_orders

Значение  \
Отдел Начало месяца РТТ                                                            
VIP   2023-02-01    1050 Роллет № 64 ряд 3 "Любушка" вход со сторон...        15   
                    107 маг."Свежий хлеб"                                      2   
                    10761 Супермаркет "Продукты № 102"                        24   
                    11990 пиццерия "Челентано"                                 2   
                    12004 маг."Еленовский"                                     2   
...                                                                          ...   
ФЕРМА 2023-02-01    9916 маг."Удача" (возле суда)                              6   
                    9939 маг. "Лилия"                                          1   
                    9942 маг. "Удача"                                          5   
                    9948 маг. "Абсолют"                                        1   
                    9952 маг. "Светлана"                                       2   

                                                                                   Атрибут  \
Отдел Начало месяца РТТ                                                                      
VIP   2023-02-01    1050 Роллет № 64 ряд 3 "Любушка" вход со сторон...  Количество заказов   
                    107 маг."Свежий хлеб"                               Количество заказов   
                    10761 Супермаркет "Продукты № 102"                  Количество заказов   
                    11990 пиццерия "Челентано"                          Количество заказов   
                    12004 маг."Еленовский"                              Количество заказов   
...                                                                                    ...   
ФЕРМА 2023-02-01    9916 маг."Удача" (возле суда)                       Количество заказов   
                    9939 маг. "Лилия"                                   Количество заказов   
                    9942 маг. "Удача"                                   Количество заказов   
                    9948 маг. "Абсолют"                                 Количество заказов   
                    9952 маг. "Светлана"                                Количество заказов   

                                                                                            Атрибут1  
Отдел Начало месяца РТТ                                                                               
VIP   2023-02-01    1050 Роллет № 64 ряд 3 "Любушка" вход со сторон...     8 и выше заказов за месяц  
                    107 маг."Свежий хлеб"                               1-2 разовых заказов за месяц  
                    10761 Супермаркет "Продукты № 102"                     8 и выше заказов за месяц  
                    11990 пиццерия "Челентано"                          1-2 разовых заказов за месяц  
                    12004 маг."Еленовский"                              1-2 разовых заказов за месяц  
...                                                                                              ...  
ФЕРМА 2023-02-01    9916 маг."Удача" (возле суда)                       5-7 разовых заказов за месяц  
                    9939 маг. "Лилия"                                   1-2 разовых заказов за месяц  
                    9942 маг. "Удача"                                   5-7 разовых заказов за месяц  
                    9948 маг. "Абсолют"                                 1-2 разовых заказов за месяц  
                    9952 маг. "Светлана"                                1-2 разовых заказов за месяц  

[4964 rows x 3 columns]

In [38]:
# Средний заказ, руб.
avg_order = sales.merge(qnt_orders, on=["Отдел", "Начало месяца"])
avg_order["Значение"] = avg_order["Значение_x"]/avg_order["Значение_y"]
avg_order.drop(columns=["Значение_x", "Атрибут_x", "Значение_y", "Атрибут_y"], inplace=True)
avg_order["Атрибут"] = "Средний заказ, руб."
avg_order

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2023-02-01,4610.413904,"Средний заказ, руб."
Главмолоко,2023-02-01,2555.645386,"Средний заказ, руб."
ФЕРМА,2023-02-01,2270.607753,"Средний заказ, руб."


In [39]:
# Средняя продажа в ТРТ, руб
avg_rtt_sale = sales.merge(qnt_rtt, on=["Отдел", "Начало месяца"])
avg_rtt_sale["Значение"] = avg_rtt_sale["Значение_x"]/avg_rtt_sale["Значение_y"]
avg_rtt_sale.drop(columns=["Значение_x", "Атрибут_x", "Значение_y", "Атрибут_y"], inplace=True)
avg_rtt_sale["Атрибут"] = "Средняя продажа в ТРТ, руб"
avg_rtt_sale

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2023-02-01,61046.323855,"Средняя продажа в ТРТ, руб"
Главмолоко,2023-02-01,15957.756553,"Средняя продажа в ТРТ, руб"
ФЕРМА,2023-02-01,12434.280551,"Средняя продажа в ТРТ, руб"


In [40]:
# Активный ассортимент

In [41]:

t.time() - time_stamp 

3.3930041790008545